# Imports

In [15]:
import pandas as pd
import ast
import pickle
from datetime import datetime

# Cleaning Functions

In [16]:
#Functions for cleaning the data for Football data
def compare_values(s):
    left, right = map(int, s.split(':'))
    if left < right:
        return 'Loss'
    elif left > right:
        return 'Win'
    else:
        return 'Draw'
    

def extract_numerical_value(text):
    try:
        # Split text by space and take the first element
        return text.split(maxsplit=1)[0]
    except:
        return None
    
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except ValueError:
        return val  # return original value if it can't be evaluated
    
def convert_extra_time(goal_time):
    if '+' in goal_time:
        base_time, extra_time = goal_time.split('+')
        return int(base_time) + int(extra_time)
    else:
        return int(goal_time)

In [17]:
# Functions to clean the data for Flight data

def convert_to_time(time_str):
    time_str = time_str.lower()  # convert to lowercase
    if "hour" in time_str:
        time_str = time_str.replace(" hours ", ":").replace(" hour ", ":").replace(" minutes", "").replace(" minute", "")
        try:
            time_obj = datetime.strptime(time_str, "%H:%M").time()
        except ValueError:
            time_obj = None  # or some default value
    else:  # if the time is in minutes only
        time_str = time_str.replace(" minutes", "").replace(" minute", "")
        if time_str.strip() == "60":
            time_obj = datetime.strptime("1:00", "%H:%M").time()
        else:
            try:
                time_obj = datetime.strptime(time_str, "%M").time()
            except ValueError:
                time_obj = None  # or some default value
    return time_obj

def extract_numerical_value_jet(text):
    try:
        # Split text by space and take the first element
        return int(text.split()[0].replace(',', ''))
    except:
        return None

# Function to split the string
def split_string(text):
    try:
        # Split text by the first bracket
        parts = text.split('(', 1)
        # First part is the value before the first bracket
        value_before_bracket = parts[0].strip()
        # Second part is the value within the bracket
        value_within_bracket = parts[1].split(')')[0].strip()
        return value_before_bracket, value_within_bracket
    except:
        return None, None

In [18]:
football = pd.read_csv('Datasets\champions_league.csv')
football['Date'] = pd.to_datetime(football['Date'])
football['Year'] = football['Date'].dt.year
football['Result'] = football['Result'].apply(extract_numerical_value)
football['Home Outcome'] = football['Result'].apply(compare_values)
football[['Home Team', 'Away Team', 'Result', 'Stage', 'Home Outcome']] = football[['Home Team', 'Away Team', 'Result', 'Stage', 'Home Outcome']].astype('string')
football[['Scoresheet_Home', 'Scoresheet_Away', 'Scorer_Home', 'Scorer_Away']] = football[['Scoresheet_Home', 'Scoresheet_Away', 'Scorer_Home', 'Scorer_Away']].applymap(safe_literal_eval)
football[['Scoresheet_Home', 'Scoresheet_Away']] = football[['Scoresheet_Home', 'Scoresheet_Away']].applymap(lambda x: [convert_extra_time(goal.strip("'")) for goal in x] if isinstance(x, list) else x)

# football.to_pickle('Datasets\champions_league_Final.pkl') # As of 11/06/2024
football

,Date,Home Team,Away Team,Result,Stage,Scoresheet_Home,Scorer_Home,Scoresheet_Away,Scorer_Away,Year,Home Outcome
0,1992-11-25,Rangers,Marseille,2:2,Group A,"[79, 81]","[Gary McSwegan, Mark Hateley]","[31, 57]","[Alen Boksic, Rudi Völler]",1992,Draw
1,1992-11-25,Club Brugge,CSKA Moscow,1:0,Group A,[16],[Daniel Amokachi],[],[],1992,Win
2,1992-12-09,CSKA Moscow,Rangers,0:1,Group A,[],[],[13],[Ian Ferguson],1992,Loss
3,1992-12-09,Marseille,Club Brugge,3:0,Group A,"[4, 10, 25]","[Franck Sauzée, Alen Boksic, Alen Boksic]",[],[],1992,Win
4,1993-03-03,Club Brugge,Rangers,1:1,Group A,[44],[Tomasz Dziubinski],[72],[Pieter Huistra],1993,Draw
...,...,...,...,...,...,...,...,...,...,...,...
3707,2024-04-30,Bayern Munich,Real Madrid,2:2,Semi Finals Leg 1,"[53, 57]","[Leroy Sané, Harry Kane]","[24, 83]","[Vinicius Junior, Vinicius Junior]",2024,Draw
3708,2024-05-01,Bor. Dortmund,Paris SG,1:0,Semi Finals Leg 1,[36],[Niclas Füllkrug],[],[],2024,Win
3709,2024-05-07,Paris SG,Bor. Dortmund,0:1,Semi Finals Leg 2,[],[],[50],[Mats Hummels],2024,Loss
3710,2024-05-08,Real Madrid,Bayern Munich,2:1,Semi Finals Leg 2,"[88, 91]","[Joselu, Joselu]",[68],[Alphonso Davies],2024,Win


In [19]:
jets = pd.read_csv('Datasets\Jets.csv')
jets['Date'] = pd.to_datetime(jets['Date'])
jets['Flight Time'] = jets['Flight Time'].apply(convert_to_time)
jets['Distance'] = jets['Distance'].apply(extract_numerical_value_jet)
jets['Fuel'] = jets['Fuel'].apply(extract_numerical_value_jet)
jets['Carbon Emissions'] = jets['Carbon Emissions'].apply(extract_numerical_value_jet)
jets['Departure'], jets['Departure Code'] = zip(*jets['Departure'].apply(split_string))
jets['Arrival'], jets['Arrival Code'] = zip(*jets['Arrival'].apply(split_string))
jets[['Departure', 'Arrival', 'Celebrity', 'Departure Code', 'Arrival Code']] = jets[['Departure', 'Arrival', 'Celebrity', 'Departure Code', 'Arrival Code']].astype('string')
jets['Flight Time'] = jets['Flight Time'].apply(lambda x: x.hour*60 + x.minute)
jets['Celebrity'] = jets['Celebrity'].replace({'Taylor Swift Plane 1': 'Taylor Swift', 'Taylor Swift Plane 2': 'Taylor Swift'})
jets['Celebrity'] = jets['Celebrity'].replace({'Michael Bloomberg Plane 1': 'Michael Bloomberg', 'Michael Bloomberg Plane 2': 'Michael Bloomberg'})


jets = jets.rename(columns={'Flight Time': 'Flight Time (Mins)', 'Carbon Emissions': 'Carbon Emissions (Metric Tons)', 'Distance': 'Distance (Miles)', 'Fuel': 'Fuel (Gallons)'})
jets

,Date,Departure,Arrival,Distance (Miles),Flight Time (Mins),Fuel (Gallons),Carbon Emissions (Metric Tons),Celebrity,Departure Code,Arrival Code
0,2023-08-26,Hollywood Burbank Airport,Nashville International Airport,1790,295,1500,20,Taylor Swift,BUR,BNA
1,2023-07-24,Paine Field Airport,Nashville International Airport,1938,250,1100,15,Taylor Swift,PAE,BNA
2,2023-07-28,Nashville International Airport,San Jose International Airport,1981,249,1000,13,Taylor Swift,BNA,SJC
3,2023-07-30,San Jose International Airport,Nashville International Airport,1864,238,1300,17,Taylor Swift,SJC,BNA
4,2023-08-02,Nashville International Airport,Hollywood Burbank Airport,1697,227,1000,14,Taylor Swift,BNA,BUR
...,...,...,...,...,...,...,...,...,...,...
1264,2023-12-13,Camarillo Airport,Van Nuys Airport,40,44,370,4,Kim Kardashian,CMA,VNY
1265,2023-07-31,Van Nuys Airport,Los Angeles International Airport,20,44,370,4,Kim Kardashian,VNY,LAX
1266,2023-07-27,Camarillo Airport,Van Nuys Airport,40,44,370,4,Kim Kardashian,CMA,VNY
1267,2023-04-30,Van Nuys Airport,McCarran International Airport,230,43,360,4,Kim Kardashian,VNY,LAS


In [ ]:
jets.dtypes

In [ ]:
print(jets.isnull().sum())
print(jets.duplicated().sum())

In [ ]:
#show null rows
jets[jets.isnull().any(axis=1)]

In [ ]:
jets = jets.dropna()
print(jets.isnull().sum())
print(jets.duplicated().sum())

In [ ]:
jets.to_pickle('Datasets\Jets-Final.pkl') # As of 30/05/2024